# TensorFlow 2 quickstart for experts

_source_: https://www.tensorflow.org/tutorials/quickstart/advanced

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Conv2D
)

## Load and prepare the MNIST dataset

In [2]:
mnist = datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

**Add a channels dimension**

In [3]:
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

**Batch and shuffle dataset**

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

2022-02-24 01:07:22.379543: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-24 01:07:22.379733: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-24 01:07:22.380940: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Create a specialized model based on tf.keras.model

In [5]:
class MyModel(Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        
        return x

**Create an instance of the model**

In [6]:
model = MyModel()

## Define a optimizer and loss function for training

In [7]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

## Create metrics to measure the loss and the accuracy of the model

In [8]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accurary")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

## Train and Test Model

**Train the model**

In [9]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

**Test the model**

In [10]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states
    test_loss.reset_states()
    test_accuracy.reset_states
    
    for images, labels in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
    
    print(
        f"Epoch {epoch + 1}, "
        f"Loss: {train_loss.result()}, "
        f"Accuracy: {train_accuracy.result() * 100}, "
        f"Test Loss: {test_loss.result()}, "
        f"Test Accuracy: {test_accuracy.result() * 100}"
    )

2022-02-24 01:07:22.842172: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-24 01:07:22.863242: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599990000 Hz


Epoch 1, Loss: 0.13648653030395508, Accuracy: 95.89666748046875, Test Loss: 0.07013160735368729, Test Accuracy: 97.63999938964844
Epoch 2, Loss: 0.04204922914505005, Accuracy: 97.30999755859375, Test Loss: 0.05730622261762619, Test Accuracy: 97.875
Epoch 3, Loss: 0.021122807636857033, Accuracy: 97.98500061035156, Test Loss: 0.051009077578783035, Test Accuracy: 98.05333709716797
Epoch 4, Loss: 0.013048380613327026, Accuracy: 98.37999725341797, Test Loss: 0.05427629128098488, Test Accuracy: 98.15999603271484
Epoch 5, Loss: 0.009324409067630768, Accuracy: 98.63899993896484, Test Loss: 0.07172208279371262, Test Accuracy: 98.19200134277344
